# Exercise 2

Recall that the *Geometric Brownian Motion* (GBM) model of stock prices is defined by
$$dS_t = \mu S_t dt + \sigma S_t dW_t$$
and it has solution

$$
S_t = S_0 \exp\left[
\left(
\mu - \frac{\sigma^2}{2}
\right) t
+ \sigma W_t
\right]
$$
## Question 1 ## 
Simulate 100 GBM sample paths, each of which has length $201$ and $S_0 = 1, \mu=0.08, \sigma=0.06$. Take the time step as $\Delta t = 1/200$.

**Hint**
1. The 200 steps of each path corresponds to $t=0.005\times1, 0.005\times2, \dots, 0.005\times200$.
2. calculate $\mu + \frac{\sigma^2}{2} = 0.08 + \frac{0.06^2}{2} = 0.818$.
3. simulate $\left(\mu - \frac{\sigma^2}{2}\right) t$ as a sequence $A$ of 200 items
   - $A_1 = 0.818 \times 1 \times 0.05$
   - $A_2 = 0.818 \times 2 \times 0.005$
   - ...
   - $A_{200} = 0.818 \times 200 \times 0.005$
4. simulate $W_t$ as another sequence of 200 items:
   1. simulate 200 standard normal random variables $x_1, x_2, \dots, x_{200}$ using scipy.stats.norm.rvs(size=200)
   2. Generate the $W_t$ sequence as
      - $W_1 = \sqrt{\Delta t} \times x_1$
      - $W_2 = \sqrt{\Delta t} \times (x_1 + x_2)$
      - ...
      - $W_{200} = \sqrt{\Delta t} \times (x_1 + x_2 + \cdots + x_{200})$

      Note: $\sqrt{\Delta t} = \sqrt{0.005} \approx 0.0707$. Use numpy.sqrt
   3. Use $\sigma = 0.06, S_0 = 1$ and simulate $S_1, S_2, \dots, S_{200}$ as
      - $S_1 = \exp(A_1 + 0.06\times W_1)$
      - $S_2 = \exp(A_2 + 0.06\times W_2)$
      - ...
      - $S_{200} = \exp(A_{200} + 0.06\times W_{200})$
        
       Use numpy.exp for the $\exp$ function.
    4. Assign $S_0 = 1$.


**Solution**:

In [1]:
import numpy as np
from scipy.stats import norm

N = 200
K = 100
S0 = 1
mu = 0.08
sigma = 0.06
Dt = 1 / N
sqrt_Dt = 1 / np.sqrt(N)

traj = np.full((N + 1, K), np.nan)
drift = (mu - sigma**2 / 2) * np.linspace(1, N, N) * Dt
for i in range(K):
    W = sqrt_Dt * np.cumsum(norm.rvs(size=N))
    traj[1:, i] = S0 * np.exp(drift + sigma * W)
    traj[0, i] = S0

## Question 2 ##
Calibrate a GBM model to each of the 100 sample paths.

**Hint**
Suppose we are calibrating the model to the sample path $S_0, S_1, S_2, \dots, S_{200}$.
1. Compute the sequence
   - $a_1 = \ln S_1 - \ln S_0$
   - $a_2 = \ln S_2 - \ln S_1$
   - ...
   - $a_{200} = \ln S_{200} - \ln S_{199}$

    Use numpy.log for the $\ln$ function.
2. Compute the mean and the standard deviation of $a_1, a_2, \dots, a_{200}$. Call the mean $\bar a$ and the standard deviation $\hat a$. Use bootstrapping for this purpose - simply adding up all the $a$'s and diving the sum by 200 will just give us $(\ln S_{200} - \ln S_0)/200$. We want to use all the $a$'s! Follow these steps:
   1. randomly choose, with replacement, 100 items from $a_1, a_2, \dots, a_{200}$. Use numpy.random.Generator.choice for this. Call the selected numbers $b_1, b_2, \dots, b_{100}$. This is a bootstrap sample. Call it bootstrap sample 1.
   2. calculate the average of bootstrap sample 1 and call the result $m_1$: $m_1 = (b_1 + b_2 + \cdots + b_{100})/100$.
   3. calculate the average of the squares of bootstrap sample 1 and call the result $q_1$: $q_1 = (b_1^2 + b_2^2 + \cdots + b_{100}^2)/100$.
   4. In the same way as we created bootstrap sample 1, create bootstrap sample 2, 3, 4, ..., 10.
   5. Calculate the average of each of the bootstrap samples and the average of the squares of each of the samples. Call them $m_2, m_3, \dots, m_{10}$ and $q_2, q_3, \dots, q_{10}$
3. Calculate $\bar a$ as
   $$\bar a = {m_1 + m_2 + \cdots + m_{10} \over 10}$$
4. Calculate $\hat a$ as
   $$\hat a = \sqrt{{q_1 + q_2 + \cdots + q_{10} \over 10} - \bar{a}^2}$$
5. Now estimate the model parameter $\sigma$ as
   $$\sigma = \frac{\hat a}{\sqrt{\Delta t}}$$
6. Estimate the model parameter $\mu$ as
   $$\mu = \frac{\bar{a}}{\Delta t} + \frac{\hat{a}^2}{2\times\Delta t}$$
7. Now the model parameters $\mu$ and $\sigma$ have been estimated, model calibration is complete.

In [8]:
class GBM:
    def __init__(self):
        self.mu = np.nan
        self.sigma = np.nan
        self.rng = np.random.default_rng()

    def calibrate(self, trajectory, Dt):
        increments = np.diff(np.log(trajectory))
        moments = [0, 0]
        n_iter = 10
        for _ in range(n_iter):
            X = self.rng.choice(increments, size=len(increments) // 2)
            moments[0] += np.mean(X) / n_iter
            moments[1] += np.mean(X**2) / n_iter
        std = np.sqrt(moments[1] - moments[0] ** 2)
        self.sigma = std / np.sqrt(Dt)
        self.mu = moments[0] / Dt + self.sigma**2 / 2


for i in range(K):
    model = GBM()
    model.calibrate(traj[:, i], Dt)
    print(f"model parameters: {model.mu}, {model.sigma}")

model parameters: -0.00920009978380331, 0.057440876414345934
model parameters: 0.09609480571468638, 0.062191014258270276
model parameters: 0.17534140299205944, 0.059089577395448636
model parameters: 0.009990607983896291, 0.06031053076051694
model parameters: 0.09110481013619458, 0.06280835420882079
model parameters: 0.08776983104510781, 0.05712591214148498
model parameters: 0.0245540861337534, 0.0560059332603707
model parameters: 0.11392170376883447, 0.05163631985968469
model parameters: 0.029676999457478356, 0.06190320047657771
model parameters: 0.22820961546354013, 0.05934090074069207
model parameters: 0.11336138931810456, 0.0614447253779296
model parameters: 0.06311903342457764, 0.06059748269162299
model parameters: 0.14022528989210054, 0.0616799482027557
model parameters: -0.06067181910021548, 0.055257975345200445
model parameters: 0.012119097451952833, 0.060881871743005696
model parameters: 0.09783268104147032, 0.055884559324266156
model parameters: -0.045973667167231114, 0.056982

## Question 3 ##
calibrate a GBM model to the provided S&P500 series.

**Solutions**: We first read the prices data into an array and then use GBM.calibrate to calibrate the model.

In [ ]:
import csv
from contextlib import closing

prices = []
with closing(open("SP500.csv")) as datafile:
    reader = csv.DictReader(datafile, fieldnames=["date", "price"], delimiter="\t")
    for row in reader:
        prices.append(float(row["price"]))
model = GBM()
model.calibrate(prices, 1 / 250)
print(f"model parameters: {model.mu}, {model.sigma}")

model parameters: 0.23201857140416315, 0.2167341726866104
